In [1]:
!pip install tensorflow opencv-python flask-ngrok numpy


In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from flask import Flask, render_template, Response
from flask_ngrok import run_with_ngrok  # To run Flask in Colab/Jupyter
from IPython.display import display, Javascript


C:\Users\Shubham Kulkarni\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# Load Haar Cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Load trained face mask detection model
mask_model = tf.keras.models.load_model("face_mask_detector.h5")


In [4]:
def detect_mask(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (224, 224))
        face = np.expand_dims(face, axis=0) / 255.0  # Normalize
        
        prediction = mask_model.predict(face)[0][0]
        
        # Calculate confidence percentage
        mask_confidence = (1 - prediction) * 100  # Higher % means wearing a mask
        no_mask_confidence = prediction * 100     # Higher % means no mask
        
        if mask_confidence > no_mask_confidence:
            label = f"Mask: {mask_confidence:.1f}%"
            color = (0, 255, 0)  # Green for Mask
        else:
            label = f"No Mask: {no_mask_confidence:.1f}%"
            color = (0, 0, 255)  # Red for No Mask

        # Add a semi-transparent background for text
        overlay = frame.copy()
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.rectangle(overlay, (x, y-40), (x+w, y), color, -1)
        alpha = 0.5  # Transparency factor
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

        # Add text with a modern font
        cv2.putText(frame, label, (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    return frame


In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = detect_mask(frame)

    cv2.imshow("Face Mask Detection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def index():
    return '''
    <html>
    <head>
        <title>Face Mask Detection</title>
        <style>
            body {
                background: url("https://source.unsplash.com/1600x900/?technology,security") no-repeat center center;
                background-size: cover;
                color: white;
                text-align: center;
                font-family: Arial, sans-serif;
            }
            .container {
                margin-top: 10%;
                background: rgba(0, 0, 0, 0.7);
                padding: 20px;
                border-radius: 10px;
                display: inline-block;
            }
            .video-container {
                border: 5px solid white;
                border-radius: 10px;
                overflow: hidden;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Face Mask Detection System</h1>
            <div class="video-container">
                <img src="/video_feed" width="100%">
            </div>
            <p>Live face mask detection with confidence percentage.</p>
        </div>
    </body>
    </html>
    '''

@app.route('/video_feed')
def video_feed():
    def generate_frames():
        cap = cv2.VideoCapture(0)
        while True:
            success, frame = cap.read()
            if not success:
                break
            else:
                frame = detect_mask(frame)
                _, buffer = cv2.imencode('.jpg', frame)
                frame = buffer.tobytes()
                yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
        cap.release()
    
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__":
    app.run()
